In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms

import torchattacks

import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os
import sys

from tqdm import tqdm

from resnet import ResNet18
from data_aug import collect_advs, AdvDataSet

TITAN RTX


In [2]:
if torch.cuda.is_available() == True:
    device = torch.device('cuda:1')
    print(torch.cuda.get_device_name())
else:
    device = torch.device('cpu')
device

TITAN RTX


device(type='cuda', index=1)

In [3]:
batch_size = 128
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def load_data():    
    transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), 
                                    transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    transform_test = transforms.Compose([transforms.ToTensor()])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_test)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    
    dataset = {'train': trainset, 'test': testset}
    data_loader = {'train': train_loader, 'test': test_loader}
    return dataset, data_loader

def load_iter(data_loader, data_type):
    bar_format = '{bar:30} {n_fmt}/{total_fmt} [{elapsed}<{remaining} {rate_fmt}] {desc}'
    
    if data_type == 'train':
        train_loader = data_loader['train']
        train_iter = tqdm(enumerate(train_loader), total=len(train_loader), unit_scale=batch_size, bar_format=bar_format)
        return train_iter
    elif data_type == 'test':
        test_loader = data_loader['test']
        test_iter = tqdm(enumerate(test_loader), total=len(test_loader), unit_scale=batch_size, bar_format=bar_format)
        return test_iter
    else:
        print('Data Error!!!')

In [4]:
def imshow(img):
    npimg = img.cpu().detach().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [5]:
def train(model):
    model.train()
    train_loss = 0
    total = 0
    correct = 0
    step = 0
    train_iter = load_iter(data_loader, 'train')
    for i, (batch, label) in train_iter:
        batch, label = batch.to(device), label.to(device)
        output = model(batch)
        
        optimizer.zero_grad()
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        acc = 100. * correct / total
        train_iter.set_description(f'[{acc:.2f}% ({correct}/{total})]', True)

In [6]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    acc = 0.
    test_iter = load_iter(data_loader, 'test')

    for i, (batch, label) in test_iter:
        batch, label = batch.to(device), label.to(device)
        output = model(batch)
        loss = loss_function(output, label)

        test_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        acc = 100. * correct / total
        test_iter.set_description(f'[{acc:.2f}%({correct}/{total})]', True)
    return acc

In [7]:
def save_model(epoch, acc, optimizer):
    global best_acc
    if acc > best_acc:
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'optimizer': optimizer.state_dict()
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/model_{epoch}.pth')
        best_acc = acc
        print('Saving Model...')

def load_model(name):
    state_dict = torch.load(f'./models/{name}.pth', map_location=device)
    model = ResNet18()
    model.to(device)
    model.load_state_dict(state_dict['model'])
    optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
#     optimizer.load_state_dict(state_dict['optimizer'])
    return model, optimizer

In [20]:
# Training
model = ResNet18()
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[25, 35])
best_acc = 0
dataset, data_loader = load_data()
for epoch in range(1, 51):
    print(f'Epoch {epoch}')
    train(model)
    test_acc = test(model)
    save_model(epoch, test_acc, optimizer)
    scheduler.step()

Files already downloaded and verified
Files already downloaded and verified


▏                              256/50048 [00:00<00:30 1627.43it/s] [10.94% (28/256)]: 

Epoch 1
torch.Size([128, 10])
torch.Size([128, 10])
torch.Size([128, 10])


▍                              768/50048 [00:00<00:28 1719.90it/s] [13.93% (107/768)]: 

torch.Size([128, 10])
torch.Size([128, 10])
torch.Size([128, 10])


...

█▉                             3328/50048 [00:01<00:25 1858.46it/s] [19.79% (684/3456)]: 

torch.Size([128, 10])
torch.Size([128, 10])
torch.Size([128, 10])


██▏                            3584/50048 [00:02<00:26 1774.67it/s] [20.09% (720/3584)]: 

torch.Size([128, 10])
torch.Size([128, 10])


KeyboardInterrupt: 

In [20]:
# Validation
model, optimizer = load_model('baseline_2')
loss_function = nn.CrossEntropyLoss()
dataset, data_loader = load_data()
test(model)

Files already downloaded and verified
Files already downloaded and verified


██████████████████████████████ 10112/10112 [00:01<00:00 5482.08it/s] [87.17%(8717/10000)]: 


87.17

## Adversarial Training

In [38]:
def adv_train(baseline, model):
    model.train()
    train_loss = 0
    total = 0
    correct = 0
    step = 0
    count = 0
    train_iter = load_iter(data_loader, 'train')
    for i, (batch, label) in train_iter:
        batch, label = batch.to(device), label.to(device)
        adv_batch = atk(batch, label)
        
        _, pred = baseline(batch).max(1)
        _, adv_pred = baseline(adv_batch).max(1)
        latent_vec = baseline.load_vec()
        
        output = model(batch)
        adv_output = model(adv_batch)
        
        
        for j, equal in enumerate(pred.eq(adv_pred)):
            dis = distance(latent_vec[j], weight[label[j].item()], bias[label[j].item()])
            if not equal and dis <= boundary_dict[label[j].item()]['bot_20']:
                count += 1
                output[j] = 0.5 * output[j] + 0.5 * adv_output[j]
    
        optimizer.zero_grad()
        loss = loss_function(output, label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        acc = 100. * correct / total
        train_iter.set_description(f'[{acc:.2f}% ({correct}/{total})] [{count}]', True)

In [39]:
# Training
model = ResNet18()
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[25, 35])
best_acc = 0
dataset, data_loader = load_data()
baseline, _ = load_model('baseline_2')
atk = torchattacks.FGSM(baseline, eps=0.5/255)
for epoch in range(1, 51):
    print(f'Epoch {epoch}')
    adv_train(baseline, model)
    test_acc = test(model)
    save_model('bot20', test_acc, optimizer)
    scheduler.step()

Files already downloaded and verified
Files already downloaded and verified


                               0/50048 [00:00<? ?it/s] 

Epoch 1


██████████████████████████████ 50048/50048 [01:28<00:00 567.30it/s] [45.30% (22649/50000)] [5068]: 
██████████████████████████████ 10112/10112 [00:01<00:00 5639.74it/s] [54.34%(5434/10000)]: 
                               0/50048 [00:00<? ?it/s] 

Saving Model...
Epoch 2


...

Epoch 49


██████████████████████████████ 50048/50048 [01:28<00:00 564.24it/s] [100.00% (50000/50000)] [5079]: 
██████████████████████████████ 10112/10112 [00:01<00:00 5668.58it/s] [87.36%(8736/10000)]: 
                               0/50048 [00:00<? ?it/s] 

Epoch 50


██████████████████████████████ 50048/50048 [01:28<00:00 564.32it/s] [100.00% (49999/50000)] [5024]: 
██████████████████████████████ 10112/10112 [00:01<00:00 5659.96it/s] [87.41%(8741/10000)]: 


Saving Model...


In [8]:
baseline, optimizer = load_model('baseline_2')
atk = torchattacks.FGSM(baseline, eps=0.5/255)
dataset, data_loader = load_data()
adv_instances = collect_advs(baseline, data_loader, atk)

Files already downloaded and verified
Files already downloaded and verified


██████████████████████████████ 50048/50048 [00:35<00:00 1415.43it/s] [# of Collected Adv Instances : 6023]: 


In [13]:
advset = AdvDataSet(adv_instances, need_perturb_label=True)
advset.slicing(42, 1000)
trainset = dataset['train']
advset_list = advset.concat_dataset(trainset)
len(advset)

6023

In [14]:
model_list = ['baseline_3']
model_list += [f'adv_{i}' for i in range(4)]
for m in model_list: 
    state_dict = torch.load(f'models/{m}.pth')
    print(m, state_dict['acc'])

baseline_3 87.66
adv_0 87.46
adv_1 86.93
adv_2 86.89
adv_3 86.74


## Hyperplane Distance

In [15]:
weight, bias = baseline.load_weight()
weight.shape, bias.shape

(torch.Size([10, 512]), torch.Size([10]))

In [16]:
def distance(a, b, c):
    a = a.cpu().detach().numpy()
    b = b.cpu().detach().numpy()
    c = c.cpu().detach().numpy()
    return np.abs(np.matmul(a, b) + c) / np.linalg.norm(b)

In [17]:
adv_loader = torch.utils.data.DataLoader(advset, batch_size=batch_size, shuffle=True)
bar_format = '{bar:30} {n_fmt}/{total_fmt} [{elapsed}<{remaining} {rate_fmt}] {desc}'
adv_iter = tqdm(enumerate(adv_loader), total=len(adv_loader), unit_scale=batch_size, bar_format=bar_format)

sort_dict = {}
boundary_dict = {}
for i in range(10):
    sort_dict[i] = []
    boundary_dict[i] = {}

new_adv_instances = []
for i, (batch, label) in adv_iter:
    batch, label = batch.to(device), label.to(device)
    output = baseline(batch)
    
    latent_vec = baseline.load_vec()
    old_label = label
    
    for j in range(batch.shape[0]):
        dis = distance(latent_vec[j], weight[old_label[j]], bias[old_label[j]])
        sort_dict[old_label[j].item()].append((dis, batch[j].cpu().detach(), old_label[j].item()))

for i in range(10):
    sort_dict[i].sort()
    idx_10 = int(len(sort_dict[i]) * 0.1)
    idx_20 = int(len(sort_dict[i]) * 0.2)
    boundary_dict[i]['top_10'] = sort_dict[i][idx_10][0]
    boundary_dict[i]['top_20'] = sort_dict[i][idx_20][0]
    boundary_dict[i]['bot_10'] = sort_dict[i][-idx_10][0]
    boundary_dict[i]['bot_20'] = sort_dict[i][-idx_20][0]
    
boundary_dict

██████████████████████████████ 6144/6144 [00:01<00:00 3631.48it/s] 


{0: {'top_10': 1.9921111,
  'top_20': 2.4569414,
  'bot_10': 4.5054946,
  'bot_20': 4.029607},
 1: {'top_10': 4.2631626,
  'top_20': 4.670611,
  'bot_10': 7.936803,
  'bot_20': 7.073451},
 2: {'top_10': 0.6874836,
  'top_20': 1.0158967,
  'bot_10': 3.1587703,
  'bot_20': 2.7136226},
 3: {'top_10': 0.7757223,
  'top_20': 1.1098003,
  'bot_10': 3.1452658,
  'bot_20': 2.6339307},
 4: {'top_10': 0.8189386,
  'top_20': 1.1776764,
  'bot_10': 3.1004822,
  'bot_20': 2.7183628},
 5: {'top_10': 1.1155912,
  'top_20': 1.5504944,
  'bot_10': 3.7928708,
  'bot_20': 3.3094227},
 6: {'top_10': 1.5036038,
  'top_20': 1.9126078,
  'bot_10': 3.988985,
  'bot_20': 3.6457815},
 7: {'top_10': 1.8043453,
  'top_20': 2.1294463,
  'bot_10': 4.8365936,
  'bot_20': 4.1259365},
 8: {'top_10': 2.617714,
  'top_20': 3.0114908,
  'bot_10': 5.2799406,
  'bot_20': 4.776397},
 9: {'top_10': 3.269793,
  'top_20': 3.720141,
  'bot_10': 6.762093,
  'bot_20': 5.922116}}

In [ ]:
advset = AdvDataSet(new_adv_instances, need_perturb_label=False)
advset.slicing(42, 3727)
trainset = dataset['train']
advset_list = advset.concat_dataset(trainset)
len(advset_list[0])

## Adversarial Attack

In [12]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    acc = 0.
    test_iter = load_iter(data_loader, 'test')

    for i, (batch, label) in test_iter:
        batch, label = batch.to(device), label.to(device)
        adv_batch = atk(batch, label)
        output = model(adv_batch)
        loss = loss_function(output, label)

        test_loss += loss.item()
        _, predicted = output.max(1)
        total += label.size(0)
        correct += predicted.eq(label).sum().item()

        acc = 100. * correct / total
        test_iter.set_description(f'[{acc:.2f}%({correct}/{total})]', True)
    return acc

In [16]:
model_list = ['baseline_2', 'fgsm', 'top10', 'top20', 'bot10', 'bot20']
loss_function = nn.CrossEntropyLoss()
dataset, data_loader = load_data()
acc = []
for m in model_list: 
    state_dict = torch.load(f'models/{m}.pth')
    model, optimizer = load_model(f'{m}')
    atk = torchattacks.FGSM(model, eps=0.5/255)
    acc.append((m, state_dict['acc'], test(model)))
acc

Files already downloaded and verified
Files already downloaded and verified


██████████████████████████████ 10112/10112 [00:06<00:00 1654.52it/s] [61.35%(6135/10000)]: 
██████████████████████████████ 10112/10112 [00:06<00:00 1672.33it/s] [69.03%(6903/10000)]: 
██████████████████████████████ 10112/10112 [00:06<00:00 1669.64it/s] [67.63%(6763/10000)]: 
██████████████████████████████ 10112/10112 [00:06<00:00 1674.64it/s] [66.04%(6604/10000)]: 
██████████████████████████████ 10112/10112 [00:06<00:00 1671.01it/s] [68.72%(6872/10000)]: 
██████████████████████████████ 10112/10112 [00:06<00:00 1671.75it/s] [66.81%(6681/10000)]: 


[('baseline_2', 87.17, 61.35),
 ('fgsm', 87.26, 69.03),
 ('top10', 87.64, 67.63),
 ('top20', 86.65, 66.04),
 ('bot10', 87.07, 68.72),
 ('bot20', 86.57, 66.81)]